# Imports

In [1]:
import numpy as np
#import pandas as pd
#import matplotlib.pyplot as plt
from tqdm import tqdm
import math

from tqdm import tqdm
import time

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from copy import deepcopy

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import train_test_split

from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.linear_model import SGDOneClassSVM
from sklearn.base import clone

from scipy.spatial.distance import cdist
from scipy.stats import ks_2samp
from scipy.optimize import minimize
from scipy.stats import wasserstein_distance

from sklearn.metrics import average_precision_score

/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/deepod/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
X_train = np.load('data/x_train.npy')
y_train = np.load('data/y_train.npy')

X_test = np.load('data/x_test.npy')
y_test = np.load('data/y_test.npy')

X = np.concatenate([X_train, X_test], axis=0)
y = np.concatenate([y_train, y_test], axis=0)

y = np.where(y == 3, 0, 1)

# Setup

In [4]:
def create_phi(normal_data, c):
    """
    Concept creation function for normal data.
    Uses k-Means clustering to partition normal data into c clusters.
    
    Args:
        normal_data (numpy array): The normal data points.
        c (int): Number of desired normal concepts.
    
    Returns:
        list of numpy arrays: List of normal clusters.
    """
    kmeans = KMeans(n_clusters=c, random_state=42)
    labels = kmeans.fit_predict(normal_data)
    
    normal_concepts = [normal_data[labels == i] for i in range(c)]
    print("Finished creating normal concepts")
    
    return normal_concepts


def create_gamma(anomaly_data, c):
    """
    Concept creation function for anomaly data.
    Uses k-Means clustering to partition anomaly data into c clusters.
    
    Args:
        anomaly_data (numpy array): The anomaly data points.
        c (int): Number of desired anomaly concepts.
    
    Returns:
        list of numpy arrays: List of anomaly clusters.
    """
    kmeans = KMeans(n_clusters=c, random_state=42)
    labels = kmeans.fit_predict(anomaly_data)
    
    anomaly_concepts = [anomaly_data[labels == i] for i in range(c)]
    print("Finished creating anomaly concepts")
    
    return anomaly_concepts
    
def match_lambda(anomaly_concepts, normal_concepts):
    """
    Matches each normal concept with the closest anomaly concept.
    Uses Euclidean distance to determine the best match.
    
    Args:
        anomaly_concepts (list of numpy arrays): List of anomaly clusters.
        normal_concepts (list of numpy arrays): List of normal clusters.
    
    Returns:
        list of tuples: Pairs of (normal_concept, matched_anomaly_concept)
    """
    pairs = []
    remaining_anomalies = anomaly_concepts.copy()

    for normal_concept in normal_concepts:
        normal_centroid = np.mean(normal_concept, axis=0)
        anomaly_centroids = [np.mean(ac, axis=0) for ac in remaining_anomalies]

        distances = cdist([normal_centroid], anomaly_centroids, metric='euclidean')[0]
        closest_idx = np.argmin(distances)

        pairs.append((normal_concept, remaining_anomalies[closest_idx]))
        remaining_anomalies.pop(closest_idx)

    print("Finished matching concept pairs")
    
    return pairs

def lifelong_roc_auc(R):
    """
    Computes the Lifelong ROC-AUC metric.
    
    Args:
        R (numpy array): NxN matrix of ROC-AUC scores, where R[i, j] is the model's 
                         performance on concept j after learning concept i.
    
    Returns:
        float: Lifelong ROC-AUC score.
    """
    N = R.shape[0]
    lower_triangular_sum = np.sum(np.tril(R))
    normalization_factor = (N * (N + 1)) / 2

    return lower_triangular_sum / normalization_factor

def BWT(R):
    """
    Computes the Backward Transfer (BWT) score.
    
    Args:
        R (numpy array): NxN results matrix.
    
    Returns:
        float: BWT score.
    """
    N = R.shape[0]
    backward_transfer = 0
    count = 0

    for i in range(1, N):
        for j in range(i):
            backward_transfer += (R[i, j] - R[j, j])
            count += 1

    return backward_transfer / count if count > 0 else 0

def FWT(R):
    """
    Computes the Forward Transfer (FWT) score.
    
    Args:
        R (numpy array): NxN results matrix.
    
    Returns:
        float: FWT score.
    """
    N = R.shape[0]
    forward_transfer = 0
    count = 0

    for i in range(N):
        for j in range(i + 1, N): 
            forward_transfer += R[i, j]
            count += 1

    return forward_transfer / count if count > 0 else 0 

def kolmogorov_smirnov_test(X_old, X_new, alpha=0.05):
    """Detect concept drift using KS-test on feature distributions."""
    
    p_values = [ks_2samp(X_old[:, i], X_new[:, i]).pvalue for i in range(X_old.shape[1])]
    return np.any(np.array(p_values) < alpha)

def histogram_binning(X, bins=25):
    """Convert sample distributions into histograms."""
    
    return np.array([np.histogram(X[:, i], bins=bins, density=True)[0] for i in range(X.shape[1])]).T

def kl_divergence(P, Q):
    """Compute KL divergence between two distributions."""
    
    P, Q = np.clip(P, 1e-10, None), np.clip(Q, 1e-10, None)  # Avoid log(0)
    return np.sum(P * np.log(P / Q))

def strategic_sample_selection(X_old, X_new, top_k=100, learning_rate=0.01, num_iterations=100):
    """
    Selects representative new samples by minimizing KL divergence.
    
    Args:
        X_old (numpy.ndarray): Old memory buffer samples.
        X_new (numpy.ndarray): Incoming new samples.
        top_k (int): Number of samples to retain.
        learning_rate (float): Step size for optimization.
        num_iterations (int): Number of optimization steps.

    Returns:
        numpy.ndarray: Selected representative new samples.
    """
    
    H_old, H_new = histogram_binning(X_old), histogram_binning(X_new)
    m_n = np.random.rand(H_new.shape[0])  

    def loss_function(m_n):
        """Computes KL divergence loss for selected samples."""
        weighted_H_new = H_new * m_n[:, np.newaxis]  
        combined_H = (H_old + weighted_H_new) / 2 
        return kl_divergence(H_new, combined_H) 

    progress_bar = tqdm(total=num_iterations, desc="Optimizing Sample Selection", position=0, leave=True)

    def callback(xk):
        progress_bar.update(1)  

    result = minimize(loss_function, m_n, method="L-BFGS-B", bounds=[(0, 1)] * len(m_n), 
                      options={"maxiter": num_iterations, "ftol": 1e-10}, callback=callback)

    progress_bar.close()

    selected_indices = np.argsort(result.x)[-top_k:]

    return X_new[selected_indices] 


def update_memory_buffer(X_old, X_new_selected, memory_size=3000):
    """Updates memory buffer using strategic forgetting."""
    updated_buffer = np.vstack((X_old, X_new_selected))  

    if updated_buffer.shape[0] > memory_size:
        updated_buffer = updated_buffer[-memory_size:]

    return updated_buffer

class HierarchicalMemory:
    def __init__(self, memory_limit=5000, pyramid_factor=2, centroids_per_concept=10):
        self.memory_limit = memory_limit
        self.pyramid_factor = pyramid_factor
        self.centroids_per_concept = centroids_per_concept
        self.memory = {}  # level: [concept1, concept2, ...]

    def add_concept(self, data, level=1):
        if level not in self.memory:
            self.memory[level] = []
        self.memory[level].append(np.array(data))
        self._summarize_memory()

    def _pyramidal_allocation(self):
        levels = sorted(self.memory.keys())
        weights = np.array([1 / (self.pyramid_factor ** (lvl - 1)) for lvl in levels])
        total_weight = weights.sum()
        allocations = (weights / total_weight) * self.memory_limit
        return {lvl: int(alloc) for lvl, alloc in zip(levels, allocations)}

    def _summarize_concept(self, concept, n_samples):
        if len(concept) <= n_samples:
            return concept
        kmeans = KMeans(n_clusters=min(self.centroids_per_concept, len(concept)), random_state=42).fit(concept)
        centroids = kmeans.cluster_centers_
        distances = np.linalg.norm(concept[:, None] - centroids, axis=2)
        closest_indices = np.argmin(distances, axis=0)
        summarized = concept[closest_indices]
        return summarized

    def _summarize_memory(self):
        allocations = self._pyramidal_allocation()
        for level, concepts in self.memory.items():
            summarized_level = []
            alloc_per_concept = max(1, allocations[level] // len(concepts))
            for concept in concepts:
                summarized = self._summarize_concept(concept, alloc_per_concept)
                summarized_level.append(summarized)
            self.memory[level] = summarized_level

    def get_all_memory(self):
        all_data = []
        for level_concepts in self.memory.values():
            for concept in level_concepts:
                all_data.append(concept)
        return np.vstack(all_data) if all_data else np.empty((0,))

def scenario_design(normal_data, anomaly_data, c):
    """
    Implements Algorithm 1 to create a lifelong learning scenario.
    
    Args:
        normal_data (numpy array): The normal data points.
        anomaly_data (numpy array): The anomaly data points.
        c (int): Number of desired concepts.
    
    Returns:
        list of tuples: List of (normal_concept, anomaly_concept) pairs forming the scenario.
    """
    normal_concepts = create_phi(normal_data, c)
    anomaly_concepts = create_gamma(anomaly_data, c)
    
    scenario = match_lambda(anomaly_concepts, normal_concepts)
    
    return scenario

def evaluation_protocol(T, E, Y, model, strategy="naive", replay_buffer_size=5000, memory_size=5000, alpha=0.05):
    """
    Implements Algorithm 2: Lifelong Learning Evaluation Protocol with multiple strategies.
    
    Args:
        T (list): Sequence of N training sets.
        E (list): Sequence of N testing sets.
        Y (list): Sequence of true labels for test sets.
        model (sklearn.base.BaseEstimator): A scikit-learn-like model instance that supports `fit` and `decision_function`.
        strategy (str): Strategy for training.
        replay_buffer_size (int): Maximum size of replay buffer if applicable
        memory_size (int): Maximum memory size if applicable
        alpha (float): KS-test threshold for drift detection.

    Returns:
        numpy array: NxN results matrix R where R[i, j] is ROC-AUC of model on E[j] after learning T[i].
    """
    N = len(T)
    R = np.zeros((N, N))  

    if strategy in ["cumulative"]:
        cumulative_data = []
    
    if strategy in ["replay"]:
        replay_buffer = []

    if strategy == "SSF":
        memory_buffer = None 

    if strategy == "hierarchical":
        h_memory = HierarchicalMemory(memory_limit=memory_size, pyramid_factor=2, centroids_per_concept=10)

    for i, Ti in tqdm(enumerate(T), desc=f"Evaluating using {strategy} strategy"):
        current_model = deepcopy(model)

        # -- NAIVE --
        if strategy == "naive":
            current_model.fit(Ti)

        # -- CUMULATIVE --
        elif strategy == "cumulative":
            cumulative_data.extend(Ti.tolist())
            current_model.fit(np.array(cumulative_data)) 

        # -- REPLAY -- 
        elif strategy == "replay":
            if replay_buffer:
                combined_data = np.vstack((np.array(replay_buffer), Ti))
            else:
                combined_data = Ti

            current_model.fit(combined_data)
            replay_buffer.extend(Ti.tolist())

            if len(replay_buffer) > replay_buffer_size:
                replay_buffer = replay_buffer[-replay_buffer_size:]
        
        # -- SSF -- 
        elif strategy == "SSF":
            if memory_buffer is None:
                memory_buffer = Ti[:memory_size]  
            else:
                drift_detected = kolmogorov_smirnov_test(memory_buffer, Ti, alpha)
                if drift_detected:
                    X_new_selected = strategic_sample_selection(memory_buffer, Ti, top_k=1000)
                    memory_buffer = update_memory_buffer(memory_buffer, X_new_selected, memory_size=memory_size)
            memory_buffer = np.unique(memory_buffer, axis=0)
            current_model.fit(memory_buffer)

        # -- HIERARCHICAL --
        elif strategy == "hierarchical":

            memory_data = h_memory.get_all_memory()
            if memory_data.size == 0:
                drift_level = 1
            else:
                drift_distances = [
                    wasserstein_distance(Ti[:, d], memory_data[:, d])
                    for d in range(Ti.shape[1])
                ]
                drift_score = np.mean(drift_distances)
                print(f"drift: {drift_score}")
                
                if drift_score < 0.05:
                    drift_level = 1
                elif drift_score < 0.1:
                    drift_level = 2
                elif drift_score < 0.2:
                    drift_level = 3
                else:
                    drift_level = 4
        
            h_memory.add_concept(Ti, level=drift_level)
            summarized_memory = h_memory.get_all_memory()
            current_model.fit(summarized_memory)

        # -- Evaluation --
        for j, ((Ej_normal, Ej_anomaly), (y_normal, y_anomaly)) in enumerate(zip(E, Y)):
            test_data = np.vstack((Ej_normal, Ej_anomaly))
            test_labels = np.hstack((y_normal, y_anomaly))  
        
            scores = -current_model.decision_function(test_data)  
            R[i, j] = roc_auc_score(test_labels, scores)

    return R


# Experiments

In [5]:
num_concepts = 5

X_normal = X[y == 0]  
X_anomaly = X[y == 1]

normal_concepts = create_phi(X_normal, num_concepts)
anomaly_concepts = create_gamma(X_anomaly, num_concepts)

concept_pairs = match_lambda(anomaly_concepts, normal_concepts)

T = []  
E = [] 
Y = []

for normal, anomaly in concept_pairs:

    normal_train, normal_test = train_test_split(normal, test_size=0.3, random_state=42)
    anomaly_train, anomaly_test = train_test_split(anomaly, test_size=0.3, random_state=42)  

    T.append(normal_train)
    E.append((normal_test, anomaly_test))

    y_normal_test = np.zeros(len(normal_test))
    y_anomaly_test = np.ones(len(anomaly_test))
    
    Y.append((y_normal_test, y_anomaly_test))

Finished creating normal concepts
Finished creating anomaly concepts
Finished matching concept pairs


# Eval

## LOF

In [10]:
R_hm = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/sklearn/neighbors/_lof.py:282: UserWarning: n_neighbors (20) is greater than the total number of samples (10). n_neighbors will be set to (n_samples - 1) for estimation.
  warnings.warn(
Evaluating using hierarchical strategy: 1it [00:16, 16.68s/it]

drift: 398.8568904220632


Evaluating using hierarchical strategy: 2it [00:27, 12.95s/it]

drift: 198.15502839106085


Evaluating using hierarchical strategy: 3it [00:34, 10.52s/it]

drift: 377.90543069776226


Evaluating using hierarchical strategy: 4it [00:40,  8.61s/it]

drift: 297.27042816559435


Evaluating using hierarchical strategy: 5it [00:52, 10.46s/it]

Lifelong ROC-AUC: 0.8072902812925056, BWT: -0.025695915608955983, FWT: 0.8816047635322224


In [26]:
R_ssf = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")


luating using SSF strategy: 0it [00:00, ?it/s]
Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 273.05it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 256.08it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 276.87it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 326.53it/s]

Evaluating using SSF strategy: 5it [00:05,  1.14s/it]

Lifelong ROC-AUC: 0.7544561432893502, BWT: -0.0039135441339650965, FWT: 0.45301001764958054


In [11]:
R_naive = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [11:18:46, ?it/s]


KeyboardInterrupt: 

In [7]:
R_cumulative = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="cumulative")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_cumulative)}, BWT: {BWT(R_cumulative)}, FWT: {FWT(R_cumulative)}")

Evaluating using cumulative strategy: 5it [02:10, 26.08s/it]

Lifelong ROC-AUC: 0.9238354062988797, BWT: -0.004499087364359666, FWT: 0.17379610432039533


In [ ]:
R_replay = evaluation_protocol(T, E, Y, LocalOutlierFactor(n_neighbors=20, novelty=True), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 0it [00:00, ?it/s]

## IF

In [11]:
R_hm = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 1it [00:19, 20.00s/it]

drift: 398.8568904220632


Evaluating using hierarchical strategy: 2it [00:34, 16.55s/it]

drift: 198.15502839106085


Evaluating using hierarchical strategy: 3it [00:44, 13.88s/it]

drift: 377.90543069776226


Evaluating using hierarchical strategy: 4it [00:53, 11.84s/it]

drift: 297.27042816559435


Evaluating using hierarchical strategy: 5it [01:08, 13.70s/it]

Lifelong ROC-AUC: 0.8155088499758174, BWT: -0.013645100218763955, FWT: 0.8445315775385265


In [27]:
R_ssf = evaluation_protocol(T, E, Y,  IsolationForest(n_estimators=100), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")


luating using SSF strategy: 0it [00:00, ?it/s]
Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 281.14it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 253.28it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 237.31it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 244.61it/s]

Evaluating using SSF strategy: 5it [00:05,  1.06s/it]

Lifelong ROC-AUC: 0.6559285630296237, BWT: 0.06868127809581366, FWT: 0.804018261100736


In [7]:
R_naive = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 5it [00:52, 10.58s/it]

Lifelong PR-AUC: 0.7788659153911035, BWT: -0.24021295357589692, FWT: 0.7917760512897282


In [9]:
R_cumulative = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="cumulative")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_cumulative)}, BWT: {BWT(R_cumulative)}, FWT: {FWT(R_cumulative)}")

Evaluating using cumulative strategy: 5it [00:07,  1.55s/it]

Lifelong ROC-AUC: 0.753897896406868, BWT: -0.0066406078135951676, FWT: 0.7719174809547474


In [8]:
R_replay = evaluation_protocol(T, E, Y, IsolationForest(n_estimators=100), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 5it [01:07, 13.46s/it]

Lifelong PR-AUC: 0.7590506633375615, BWT: -0.27065328400264504, FWT: 0.8040770717615044


## SGDOCSVM

In [12]:
R_hm = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 1it [00:13, 13.54s/it]

drift: 398.8568904220632


Evaluating using hierarchical strategy: 2it [00:20,  9.93s/it]

drift: 198.15502839106085


Evaluating using hierarchical strategy: 3it [00:24,  7.18s/it]

drift: 377.90543069776226


Evaluating using hierarchical strategy: 4it [00:26,  4.94s/it]

drift: 297.27042816559435


Evaluating using hierarchical strategy: 5it [00:34,  6.82s/it]

Lifelong ROC-AUC: 0.711944549167925, BWT: -0.04644052916489031, FWT: 0.468725564548585


In [28]:
R_ssf = evaluation_protocol(T, E, Y,  SGDOneClassSVM(), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")


luating using SSF strategy: 0it [00:00, ?it/s]
Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 288.13it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 307.55it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 306.85it/s]

Optimizing Sample Selection:   1%|          | 1/100 [00:00<00:00, 271.76it/s]

Evaluating using SSF strategy: 5it [00:02,  2.14it/s]

Lifelong ROC-AUC: 0.790790866239933, BWT: -0.004091332504534584, FWT: 0.5467376505389003


In [9]:
R_naive = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 5it [00:14,  2.89s/it]

Lifelong PR-AUC: 0.46159136314423416, BWT: -0.21579492490317728, FWT: 0.6273475196471586


In [17]:
R_cumulative = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="cumulative")
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_cumulative)}, BWT: {BWT(R_cumulative)}, FWT: {FWT(R_cumulative)}")

Evaluating using cumulative strategy: 5it [00:04,  1.03it/s]


Lifelong ROC-AUC: 0.597799790906325, BWT: -0.215598947769658, FWT: 0.6579349113663381


In [10]:
R_replay = evaluation_protocol(T, E, Y, SGDOneClassSVM(), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

Evaluating using replay strategy: 5it [00:28,  5.70s/it]

Lifelong PR-AUC: 0.4944899524729847, BWT: -0.21646644626189934, FWT: 0.5427823493460701


## SLAD

In [6]:
from deepod.models.tabular import SLAD

In [ ]:
R_hm = evaluation_protocol(T, E, Y, SLAD(), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 33, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33]


/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


epoch  1, training loss: 1.044842, time: 0.4s
epoch 10, training loss: 0.625887, time: 0.0s
epoch 20, training loss: 0.594825, time: 0.0s
epoch 30, training loss: 0.575092, time: 0.0s
epoch 40, training loss: 0.576783, time: 0.0s
epoch 50, training loss: 0.560773, time: 0.0s
epoch 60, training loss: 0.558940, time: 0.0s
epoch 70, training loss: 0.573671, time: 0.0s
epoch 80, training loss: 0.529655, time: 0.0s
epoch 90, training loss: 0.586903, time: 0.0s
epoch100, training loss: 0.551060, time: 0.0s
Start Inference on the training data...


In [7]:
R_ssf = evaluation_protocol(T, E, Y, SLAD(epochs=10), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Evaluating using SSF strategy: 0it [00:00, ?it/s]/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 33, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33]
epoch  1, training loss: 0.501681, time: 4.0s
epoch 10, training loss: 0.313463, time: 0.2s
Start Inference on the training data...


Optimizing Sample Selection:   8%|▊         | 8/100 [00:00<00:00, 610.68it/s]
/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 33, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33]
epoch  1, training loss: 0.825984, time: 0.2s
epoch 10, training loss: 0.414694, time: 0.2s
Start Inference on the training data...


Optimizing Sample Selection:  11%|█         | 11/100 [00:00<00:00, 714.18it/s]
/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


...

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 33, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33]
epoch  1, training loss: 0.667521, time: 0.3s
epoch 10, training loss: 0.431774, time: 0.3s
Start Inference on the training data...


Optimizing Sample Selection:  10%|█         | 10/100 [00:00<00:00, 516.40it/s]
/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2897: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/venv/lib/python3.11/site-packages/numpy/lib/function_base.py:2898: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]


Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 33, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33]
epoch  1, training loss: 0.658839, time: 0.3s
epoch 10, training loss: 0.510152, time: 0.3s
Start Inference on the training data...


Evaluating using SSF strategy: 5it [1:37:56, 1175.38s/it]

Lifelong ROC-AUC: 0.3103566662499718, BWT: 0.21562569546654134, FWT: 0.011228514161569971


In [7]:
R_naive = evaluation_protocol(T, E, Y, SLAD(epochs=10), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
unified size: 128, subspace pool size: 33, ensemble size: 20
len pool: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24
 25 26 27 28 29 30 31 32 33]


/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/deepod2/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:3045: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/home/jovyan/Desktop/AdversarialAttacks/_SSL/mqtt/deepod2/lib/python3.11/site-packages/numpy/lib/_function_base_impl.py:3046: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
Evaluating using naive strategy: 0it [00:00, ?it/s]


RuntimeError: Numpy is not available

In [ ]:
R_replay = evaluation_protocol(T, E, Y, SLAD(epochs=10), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

# ICL

In [6]:
from deepod.models.tabular import ICL

In [8]:
R_hm = evaluation_protocol(T, E, Y, ICL(epochs=100), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=31, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2, out_features=50, bias=Fa


testing: 100%|██████████| 1/1 [00:00<00:00, 10.37it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 459.30it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 560.66it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 627.98it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 633.71it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 634.06it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 638.84it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 638.90it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 639.00it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 639.71it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 640.12it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 639.25it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 641.61it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 643.47it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 640.14it/s]

testing: 100%|██████████| 10537/10537 [00:16<00:00, 635.90it/s]

te

drift: 398.8568904220632
Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=31, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2,


testing: 100%|██████████| 1/1 [00:00<00:00, 528.85it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 538.98it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 545.85it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 634.93it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 628.77it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 635.97it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 635.70it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 638.98it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 638.10it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 638.73it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 639.09it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 641.36it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 641.67it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 641.80it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 641.64it/s]

testing: 100%|██████████| 10537/10537 [00:16<00:00, 634.23it/s]

t

...

drift: 377.90543069776226
Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=31, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2


testing: 100%|██████████| 1/1 [00:00<00:00, 505.70it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 515.46it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 530.79it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 636.30it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 634.31it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 636.67it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 640.14it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 633.08it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 639.36it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 640.72it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 641.33it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 640.95it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 641.36it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 642.52it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 642.53it/s]

testing: 100%|██████████| 10537/10537 [00:16<00:00, 638.76it/s]

t

drift: 297.27042816559435
Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=31, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2


testing: 100%|██████████| 1/1 [00:00<00:00, 504.24it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 516.54it/s]

testing: 100%|██████████| 1/1 [00:00<00:00, 503.82it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 636.50it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 636.89it/s]

testing: 100%|██████████| 30715/30715 [00:48<00:00, 634.63it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 640.55it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 635.79it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 638.67it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 641.50it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 638.73it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 640.55it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 641.29it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 642.99it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 642.38it/s]

testing: 100%|██████████| 10537/10537 [00:16<00:00, 638.02it/s]

t

Lifelong ROC-AUC: 0.08254430546472566, BWT: 0.0603046965289877, FWT: 0.3023565450702955


In [7]:
R_ssf = evaluation_protocol(T, E, Y, ICL(epochs=10), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Evaluating using SSF strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=31, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2, out_features=50, bias=Fa


testing: 100%|██████████| 12/12 [00:00<00:00, 87.35it/s][A

testing: 100%|██████████| 12/12 [00:00<00:00, 673.09it/s]

testing: 100%|██████████| 12/12 [00:00<00:00, 575.87it/s]

testing: 100%|██████████| 30715/30715 [00:46<00:00, 663.61it/s]

testing: 100%|██████████| 30715/30715 [00:46<00:00, 666.94it/s]

testing: 100%|██████████| 30715/30715 [00:46<00:00, 666.53it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 670.55it/s]

testing: 100%|██████████| 10065/10065 [00:14<00:00, 673.06it/s]

testing: 100%|██████████| 10065/10065 [00:14<00:00, 672.49it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 673.91it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 671.57it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 672.44it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 674.08it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 676.18it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 674.76it/s]

testing: 100%|██████████| 10537/10537 [00:15<00:00, 668.30i

Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=31, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2, out_features=50, bias=Fa


testing: 100%|██████████| 12/12 [00:00<00:00, 672.69it/s]

testing: 100%|██████████| 12/12 [00:00<00:00, 675.65it/s]

testing: 100%|██████████| 12/12 [00:00<00:00, 673.03it/s]

testing: 100%|██████████| 30715/30715 [00:45<00:00, 668.54it/s]

testing: 100%|██████████| 30715/30715 [00:45<00:00, 668.56it/s]

testing: 100%|██████████| 30715/30715 [00:45<00:00, 670.31it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 669.67it/s]

testing: 100%|██████████| 10065/10065 [00:14<00:00, 672.16it/s]

testing: 100%|██████████| 10065/10065 [00:14<00:00, 674.16it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 673.70it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 675.31it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 672.79it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 674.80it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 674.32it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 675.58it/s]

testing: 100%|██████████| 10537/10537 [00:15<00:00, 668.80it

...

Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=31, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2, out_features=50, bias=Fa


testing: 100%|██████████| 13/13 [00:00<00:00, 657.26it/s]

testing: 100%|██████████| 13/13 [00:00<00:00, 672.81it/s]

testing: 100%|██████████| 13/13 [00:00<00:00, 667.04it/s]

testing: 100%|██████████| 30715/30715 [00:45<00:00, 669.62it/s]

testing: 100%|██████████| 30715/30715 [00:45<00:00, 668.81it/s]

testing: 100%|██████████| 30715/30715 [00:45<00:00, 670.08it/s]

testing: 100%|██████████| 10065/10065 [00:15<00:00, 668.48it/s]

testing: 100%|██████████| 10065/10065 [00:14<00:00, 673.08it/s]

testing: 100%|██████████| 10065/10065 [00:14<00:00, 673.93it/s][A

testing: 100%|██████████| 4776/4776 [00:07<00:00, 674.32it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 673.36it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 661.86it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 675.15it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 673.00it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 676.20it/s]

testing: 100%|██████████| 10537/10537 [00:15<00:00, 672.21

Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=31, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2, out_features=50, bias=Fa


testing: 100%|██████████| 13/13 [00:00<00:00, 669.80it/s]

testing: 100%|██████████| 13/13 [00:00<00:00, 654.27it/s]

testing: 100%|██████████| 13/13 [00:00<00:00, 672.62it/s]

testing: 100%|██████████| 30715/30715 [00:45<00:00, 669.62it/s]

testing: 100%|██████████| 30715/30715 [00:45<00:00, 669.22it/s]

testing: 100%|██████████| 30715/30715 [00:45<00:00, 669.82it/s]

testing: 100%|██████████| 10065/10065 [00:14<00:00, 674.88it/s]

testing: 100%|██████████| 10065/10065 [00:14<00:00, 674.35it/s][A

testing: 100%|██████████| 10065/10065 [00:14<00:00, 674.35it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 675.10it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 675.25it/s]

testing: 100%|██████████| 4776/4776 [00:07<00:00, 674.19it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 673.56it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 675.45it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 674.03it/s]

testing: 100%|██████████| 10537/10537 [00:15<00:00, 668.63

Lifelong ROC-AUC: 0.07980195498392721, BWT: 0.015148636937932641, FWT: 0.08914322453508312


In [ ]:
R_naive = evaluation_protocol(T, E, Y, ICL(epochs=10), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

Evaluating using naive strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 3
kernel size: 2
ICLNet(
  (enc_f_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=31, out_features=100, bias=False)
        (act_layer): Tanh()
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (bn_layer): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
      )
    )
  )
  (enc_g_net): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=2, out_features=50, bias=Fa


testing: 100%|██████████| 70712/70712 [02:01<00:00, 583.42it/s]

testing:  70%|███████   | 49554/70712 [01:23<00:37, 566.77it/s]

In [ ]:
R_replay = evaluation_protocol(T, E, Y, ICL(epochs=10), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

# RCA

In [ ]:
from deepod.models.tabular import RCA

In [10]:
R_hm = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 1098.04it/s]

100%|██████████| 10/10 [04:42<00:00, 28.24s/it]

100%|██████████| 10/10 [01:32<00:00,  9.25s/it]

100%|██████████| 10/10 [00:43<00:00,  4.39s/it]

100%|██████████| 10/10 [00:04<00:00,  2.05it/s]

100%|██████████| 10/10 [01:36<00:00,  9.68s/it]
Evaluating using hierarchical strategy: 1it [08:56, 536.71s/it]

drift: 398.8568904220632
Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, o


100%|██████████| 10/10 [00:00<00:00, 1022.48it/s]

100%|██████████| 10/10 [04:41<00:00, 28.20s/it]

100%|██████████| 10/10 [01:32<00:00,  9.25s/it]

100%|██████████| 10/10 [00:43<00:00,  4.39s/it]

100%|██████████| 10/10 [00:04<00:00,  2.05it/s]

100%|██████████| 10/10 [01:36<00:00,  9.67s/it]
Evaluating using hierarchical strategy: 2it [17:46, 532.46s/it]

...

drift: 377.90543069776226
Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, 


100%|██████████| 10/10 [00:00<00:00, 981.19it/s]

100%|██████████| 10/10 [04:40<00:00, 28.03s/it]

100%|██████████| 10/10 [01:31<00:00,  9.18s/it]

100%|██████████| 10/10 [00:43<00:00,  4.38s/it]

100%|██████████| 10/10 [00:04<00:00,  2.04it/s]

100%|██████████| 10/10 [01:36<00:00,  9.69s/it]
Evaluating using hierarchical strategy: 4it [35:10, 525.55s/it]

drift: 297.27042816559435
Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, 


100%|██████████| 10/10 [00:00<00:00, 948.77it/s]

100%|██████████| 10/10 [04:42<00:00, 28.28s/it]

100%|██████████| 10/10 [01:32<00:00,  9.25s/it]

100%|██████████| 10/10 [00:43<00:00,  4.39s/it]

100%|██████████| 10/10 [00:04<00:00,  2.04it/s]

100%|██████████| 10/10 [01:36<00:00,  9.69s/it]
Evaluating using hierarchical strategy: 5it [44:01, 528.30s/it]

Lifelong ROC-AUC: 0.4234612656150892, BWT: 0.0027778110432063047, FWT: 0.13952288609745678


In [10]:
R_ssf = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Evaluating using SSF strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 95.71it/s]

100%|██████████| 10/10 [06:33<00:00, 39.40s/it]

100%|██████████| 10/10 [01:28<00:00,  8.86s/it]

100%|██████████| 10/10 [00:42<00:00,  4.21s/it]

100%|██████████| 10/10 [00:04<00:00,  2.14it/s]

Optimizing Sample Selection:   8%|▊         | 8/100 [00:00<00:00, 536.06it/s]


Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 94.50it/s]

100%|██████████| 10/10 [04:31<00:00, 27.15s/it]

100%|██████████| 10/10 [01:28<00:00,  8.89s/it]

100%|██████████| 10/10 [00:42<00:00,  4.22s/it]

100%|██████████| 10/10 [00:04<00:00,  2.13it/s]

Optimizing Sample Selection:  11%|█         | 11/100 [00:00<00:00, 670.05it/s]


...

Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 87.73it/s][A

100%|██████████| 10/10 [04:31<00:00, 27.11s/it]

100%|██████████| 10/10 [01:29<00:00,  8.90s/it]

100%|██████████| 10/10 [00:42<00:00,  4.21s/it]

100%|██████████| 10/10 [00:04<00:00,  2.13it/s]

Optimizing Sample Selection:   9%|▉         | 9/100 [00:00<00:00, 530.02it/s]


Start Training...
ensemble size: 1
RCANet(
  (enc1): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (2): LinearBlock(
        (linear): Linear(in_features=50, out_features=128, bias=False)
        (act_layer): Identity()
      )
    )
  )
  (enc2): MLPnet(
    (network): Sequential(
      (0): LinearBlock(
        (linear): Linear(in_features=33, out_features=100, bias=False)
        (act_layer): LeakyReLU(negative_slope=0.01)
        (dropout_layer): Dropout(p=0.5, inplace=False)
      )
      (1): LinearBlock(
        (linear): Linear(in_features=100, out_features=50, bias=Fals


100%|██████████| 10/10 [00:00<00:00, 87.00it/s][A

100%|██████████| 10/10 [04:31<00:00, 27.17s/it]

100%|██████████| 10/10 [01:28<00:00,  8.87s/it]

100%|██████████| 10/10 [00:42<00:00,  4.22s/it]

100%|██████████| 10/10 [00:04<00:00,  2.13it/s]

100%|██████████| 10/10 [01:33<00:00,  9.31s/it]
Evaluating using SSF strategy: 5it [44:09, 529.82s/it]

Lifelong ROC-AUC: 0.3282630514484718, BWT: -0.020036791655979554, FWT: 0.20085286353009826


In [ ]:
R_naive = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

In [ ]:
R_replay = evaluation_protocol(T, E, Y, RCA(epochs=10), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")

# RDP

In [ ]:
from deepod.models.tabular import RDP

In [12]:
R_hm = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="hierarchical", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_hm)}, BWT: {BWT(R_hm)}, FWT: {FWT(R_hm)}")

Evaluating using hierarchical strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=33, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.7s
epoch 10, training loss: 1.182584, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 741.31it/s]

testing: 100%|██████████| 30715/30715 [00:29<00:00, 1039.54it/s]

testing: 100%|██████████| 10065/10065 [00:09<00:00, 1047.04it/s]

testing: 100%|██████████| 4776/4776 [00:04<00:00, 1034.16it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 1037.63it/s]

testing: 100%|██████████| 10537/10537 [00:10<00:00, 1039.07it/s]
Evaluating using hierarchical strategy: 1it [01:09, 69.52s/it]

drift: 398.8568904220632
Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=33, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.0s
epoch 10, training loss: 62.041412, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 261.49it/s]

testing: 100%|██████████| 30715/30715 [00:41<00:00, 744.90it/s]

testing: 100%|██████████| 10065/10065 [00:13<00:00, 758.07it/s]

testing: 100%|██████████| 4776/4776 [00:06<00:00, 781.28it/s] 

testing: 100%|██████████| 539/539 [00:01<00:00, 483.14it/s]

testing: 100%|██████████| 10537/10537 [00:10<00:00, 1017.94it/s]
Evaluating using hierarchical strategy: 2it [02:33, 78.02s/it]

...

drift: 377.90543069776226
Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=33, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.0s
epoch 10, training loss: 104.846001, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 344.64it/s]

testing: 100%|██████████| 30715/30715 [00:30<00:00, 1017.80it/s][A

testing: 100%|██████████| 10065/10065 [00:09<00:00, 1039.59it/s]

testing: 100%|██████████| 4776/4776 [00:04<00:00, 1040.57it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 1037.02it/s]

testing: 100%|██████████| 10537/10537 [00:10<00:00, 1041.59it/s]
Evaluating using hierarchical strategy: 4it [04:34, 66.19s/it]

drift: 297.27042816559435
Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=33, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.000000, time: 0.0s
epoch 10, training loss: 37.008541, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 1/1 [00:00<00:00, 737.40it/s]

testing: 100%|██████████| 30715/30715 [00:29<00:00, 1048.90it/s]

testing: 100%|██████████| 10065/10065 [00:09<00:00, 1054.50it/s][A

testing: 100%|██████████| 4776/4776 [00:04<00:00, 1052.50it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 1051.09it/s]

testing: 100%|██████████| 10537/10537 [00:09<00:00, 1054.95it/s]
Evaluating using hierarchical strategy: 5it [05:38, 67.72s/it]

Lifelong ROC-AUC: 0.2109363524529993, BWT: -0.05559681466074164, FWT: 0.16230645451674172


In [ ]:
R_ssf = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="SSF", memory_size=5000, alpha=0.05)
print(f"Lifelong ROC-AUC: {lifelong_roc_auc(R_ssf)}, BWT: {BWT(R_ssf)}, FWT: {FWT(R_ssf)}")

Evaluating using SSF strategy: 0it [00:00, ?it/s]

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=33, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 0.074373, time: 0.0s
epoch 10, training loss: 0.000723, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 12/12 [00:00<00:00, 1071.80it/s]

testing: 100%|██████████| 30715/30715 [00:28<00:00, 1075.80it/s]

testing: 100%|██████████| 10065/10065 [00:09<00:00, 1079.84it/s]

testing: 100%|██████████| 4776/4776 [00:04<00:00, 1079.72it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 1075.93it/s]

Optimizing Sample Selection:   8%|▊         | 8/100 [00:00<00:00, 560.66it/s]


Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=33, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 7.727429, time: 0.0s
epoch 10, training loss: 0.317424, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 12/12 [00:00<00:00, 1066.03it/s]

testing: 100%|██████████| 30715/30715 [00:28<00:00, 1074.98it/s]

testing: 100%|██████████| 10065/10065 [00:09<00:00, 1080.86it/s]

testing: 100%|██████████| 4776/4776 [00:04<00:00, 1081.99it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 1076.41it/s]

Optimizing Sample Selection:  11%|█         | 11/100 [00:00<00:00, 694.77it/s]


...

Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=33, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 11.480359, time: 0.0s
epoch 10, training loss: 1.992806, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 13/13 [00:00<00:00, 1051.35it/s]

testing: 100%|██████████| 30715/30715 [00:28<00:00, 1077.23it/s]

testing: 100%|██████████| 10065/10065 [00:09<00:00, 1069.25it/s][A

testing: 100%|██████████| 4776/4776 [00:04<00:00, 1077.74it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 1076.55it/s]

Optimizing Sample Selection:   9%|▉         | 9/100 [00:00<00:00, 556.56it/s]


Start Training...
ensemble size: 1
MLPnet(
  (network): Sequential(
    (0): LinearBlock(
      (linear): Linear(in_features=33, out_features=100, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (1): LinearBlock(
      (linear): Linear(in_features=100, out_features=50, bias=False)
      (act_layer): LeakyReLU(negative_slope=0.01)
    )
    (2): LinearBlock(
      (linear): Linear(in_features=50, out_features=128, bias=False)
      (act_layer): Identity()
    )
  )
)
epoch  1, training loss: 13.060379, time: 0.0s
epoch 10, training loss: 1.052357, time: 0.0s
Start Inference on the training data...



testing: 100%|██████████| 13/13 [00:00<00:00, 1032.10it/s]

testing: 100%|██████████| 30715/30715 [00:28<00:00, 1073.90it/s]

testing: 100%|██████████| 10065/10065 [00:09<00:00, 1081.12it/s]

testing: 100%|██████████| 4776/4776 [00:04<00:00, 1075.33it/s]

testing: 100%|██████████| 539/539 [00:00<00:00, 1076.64it/s]

testing: 100%|██████████| 10537/10537 [00:09<00:00, 1079.49it/s]
Evaluating using SSF strategy: 5it [04:48, 57.76s/it]

Lifelong ROC-AUC: 0.12294627241713126, BWT: 0.03032751156809607, FWT: 0.013198038793983985


In [ ]:
R_naive = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="naive")
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_naive)}, BWT: {BWT(R_naive)}, FWT: {FWT(R_naive)}")

In [ ]:
R_replay = evaluation_protocol(T, E, Y, RDP(epochs=10), strategy="replay", replay_buffer_size=5000)
print(f"Lifelong PR-AUC: {lifelong_roc_auc(R_replay)}, BWT: {BWT(R_replay)}, FWT: {FWT(R_replay)}")